<a href="https://colab.research.google.com/github/du-hr/CrowdCounter/blob/main/Crowd_Counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECSE 415 Course Project - Counting People in a Shopping Mall


> Final Project of ECSE 415 (Fall 2020) @ McGill University


> Authors (G32): Haoran Du (260776911), Daniel Aird (260865951)





## **0. Initialization (First run?-> Run, restart runtime, and run again)**

In [ ]:
#Initialization of Detectron2 + restart runtime
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
#exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [1]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [3]:
import cv2
import numpy as np
import scipy as sci
import pandas as pd
from google.colab import drive
# ignore the follwoing line if running locally
drive.mount('/content/drive')
# make path = './' if running locally
path = '/content/drive/My Drive/frames/frames/'

#%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **1. Datasets**

### **1.1 Import Data**

Instructions: If this is your first time running the code, uncomment Section 1.1 and Section 1.2 and import the dataset from the images. Then, download the new datasetGray.npz file so you can use that next time. If you already have the datasetGray.npz file saved, just upload it to the session and load it using Section 1.3.

In [4]:
#images = []

# load seq_000001.jpg to seq_000009.jpg
#for i in range(9):
#  path_i = path + 'seq_00000' + str(i+1) +'.jpg'
#  images.append(cv2.cvtColor(cv2.imread(path_i), cv2.COLOR_BGR2GRAY))

# load seq_000010.jpg to seq_000099.jpg
#for i in range(9,99):
#  path_i = path + 'seq_0000' + str(i+1) +'.jpg'
#  images.append(cv2.cvtColor(cv2.imread(path_i), cv2.COLOR_BGR2GRAY))

# load seq_000100.jpg to seq_000999.jpg
#for i in range(99,999):
#  path_i = path + 'seq_000' + str(i+1) +'.jpg'
#  images.append(cv2.cvtColor(cv2.imread(path_i), cv2.COLOR_BGR2GRAY))

# load seq_001000.jpg to seq_002000.jpg
#for i in range(999,2000):
#  path_i = path + 'seq_00' + str(i+1) +'.jpg'
#  images.append(cv2.cvtColor(cv2.imread(path_i), cv2.COLOR_BGR2GRAY))

### **1.2 Save Dataset**

In [5]:
#datasetGray = np.asarray(images)
#np.savez('datasetGray', datasetGray)

### **1.3 Load Saved Dataset**

In [ ]:
#Read the datasetGray ndarray instead of importing every image
datasetGray = np.load('./datasetGray')

## **2. Existing Person Detection Models**




In [ ]:
#First download an image from the COCO dataset
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
im = cv2.imread("./input.jpg")
cv2_imshow(im)

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.DEVICE='cpu'
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

In [ ]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
outputs['instances'].pred_classes.tolist()

## **3. SVM Model**

## **3. Performance**